# Kerakli kutubxonalarni chaqiramiz

In [ ]:
import pandas as pd
import numpy as np

from joblib import dump
import os

from sklearn.model_selection import train_test_split, KFold,cross_val_score
from sklearn.feature_selection import RFE

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from xgboost import XGBClassifier

Xgb_Clas = XGBClassifier()
N_neig = NearestNeighbors()
DT_Clas = DecisionTreeClassifier()
RF_Clas = RandomForestClassifier()
SV_Clas = SVC()
LR_Clas = LogisticRegression()

from sklearn.metrics import accuracy_score

# datasetni yuklaymiz

In [2]:
df = pd.read_csv(r'C:\Users\bunyo\OneDrive\Desktop\git_project\data\preprocessed_data\preprocessed_data.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           25000 non-null  float64
 1   Location       25000 non-null  float64
 2   MinTemp        25000 non-null  float64
 3   MaxTemp        25000 non-null  float64
 4   Rainfall       25000 non-null  float64
 5   Evaporation    25000 non-null  float64
 6   Sunshine       25000 non-null  float64
 7   WindGustDir    25000 non-null  float64
 8   WindGustSpeed  25000 non-null  float64
 9   WindDir9am     25000 non-null  float64
 10  WindDir3pm     25000 non-null  float64
 11  WindSpeed9am   25000 non-null  float64
 12  WindSpeed3pm   25000 non-null  float64
 13  Humidity9am    25000 non-null  float64
 14  Humidity3pm    25000 non-null  float64
 15  Pressure9am    25000 non-null  float64
 16  Pressure3pm    25000 non-null  float64
 17  Cloud9am       25000 non-null  float64
 18  Cloud3

# filter orqali yaxshi featurelarni olamiz

In [9]:
corr_df = df.corr()['RainTomorrow']
corr_df

Date            -0.007687
Location        -0.000092
MinTemp          0.090613
MaxTemp         -0.150775
Rainfall         0.245760
Evaporation     -0.077323
Sunshine        -0.316011
WindGustDir      0.052806
WindGustSpeed    0.222305
WindDir9am       0.033265
WindDir3pm       0.029994
WindSpeed9am     0.090808
WindSpeed3pm     0.079471
Humidity9am      0.262566
Humidity3pm      0.432617
Pressure9am     -0.238477
Pressure3pm     -0.220121
Cloud9am         0.247934
Cloud3pm         0.291763
Temp9am         -0.018161
Temp3pm         -0.182194
RainTomorrow     1.000000
Name: RainTomorrow, dtype: float64

In [37]:
corr_df1 = corr_df[abs(corr_df) > 0.2]
corr_df1

Rainfall         0.245760
Sunshine        -0.316011
WindGustSpeed    0.222305
Humidity9am      0.262566
Humidity3pm      0.432617
Pressure9am     -0.238477
Pressure3pm     -0.220121
Cloud9am         0.247934
Cloud3pm         0.291763
RainTomorrow     1.000000
Name: RainTomorrow, dtype: float64

In [38]:
df1 = df[corr_df1.index]

# Qolgan dataset bilan boshqa dataset yasaymiz

In [ ]:
df1['AverageHumidity'] = (df1['Humidity9am'] + df1['Humidity3pm']) / 2
df1['AvgPressure'] = (df1['Pressure9am'] + df1['Pressure3pm']) /2
df1['AvrCloud'] = (df1['Cloud9am'] + df1['Cloud3pm']) / 2

In [49]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rainfall         25000 non-null  float64
 1   Sunshine         25000 non-null  float64
 2   WindGustSpeed    25000 non-null  float64
 3   Humidity9am      25000 non-null  float64
 4   Humidity3pm      25000 non-null  float64
 5   Pressure9am      25000 non-null  float64
 6   Pressure3pm      25000 non-null  float64
 7   Cloud9am         25000 non-null  float64
 8   Cloud3pm         25000 non-null  float64
 9   RainTomorrow     25000 non-null  float64
 10  AverageHumidity  25000 non-null  float64
 11  AvgPressure      25000 non-null  float64
 12  AvrCloud         25000 non-null  float64
dtypes: float64(13)
memory usage: 2.5 MB


In [51]:
df1.corr()['RainTomorrow']

Rainfall           0.245760
Sunshine          -0.316011
WindGustSpeed      0.222305
Humidity9am        0.262566
Humidity3pm        0.432617
Pressure9am       -0.238477
Pressure3pm       -0.220121
Cloud9am           0.247934
Cloud3pm           0.291763
RainTomorrow       1.000000
AverageHumidity    0.307166
AvgPressure       -0.231589
AvrCloud           0.306574
Name: RainTomorrow, dtype: float64

# Datasetni ajratamiz

In [52]:
x = df1.drop('RainTomorrow',axis=1)
y = df['RainTomorrow']

In [72]:
y = y.astype(int)

In [73]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.25, random_state=99)

# Gradient Boosting

In [ ]:
grBoost = GradientBoostingClassifier(n_estimators=199, max_depth=4, random_state=99)

Rfe = RFE(grBoost, n_features_to_select=10)
Rfe.fit(x_train, y_train)
SelectedFeatures = x_train.columns[Rfe.support_]

grBoost.fit(x_train[SelectedFeatures], y_train)
pred1 = grBoost.predict(x_test[SelectedFeatures])

print(accuracy_score(y_test,pred1))

0.84496


# XGBoost Classifier

In [76]:
Rfe = RFE(Xgb_Clas, n_features_to_select=10)
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

Xgb_Clas.fit(x_train[selected_features], y_train)
pred2 = Xgb_Clas.predict(x_test[selected_features])

print(accuracy_score(y_test, pred2))

0.84192


# Random Forest

In [ ]:
Rfe = RFE(RF_Clas, n_features_to_select=10) 
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

RF_Clas.fit(x_train[selected_features], y_train)
pred3 = RF_Clas.predict(x_test[selected_features])

print(accuracy_score(y_test, pred3))

0.84416


# Decision Tree

In [79]:
Rfe = RFE(DT_Clas, n_features_to_select=10)
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

DT_Clas.fit(x_train[selected_features], y_train)
pred4 = DT_Clas.predict(x_test[selected_features])

print(accuracy_score(y_test, pred4))

0.7744


# SVM

In [89]:
SV_Clas.fit(x_train,y_train)

SVR()

In [90]:
pred5 = SV_Clas.predict(x_test)


In [91]:
pred5

array([-0.12142094,  0.44813026, -0.02824374, ...,  0.09675518,
       -0.04078583,  0.08717734])

# Linear Model

In [92]:
LR_Clas.fit(x_train,y_train)

LogisticRegression()

In [95]:
pred6 = LR_Clas.predict(x_test)
print(accuracy_score(y_test, pred6))

0.84016


# tabulate

In [97]:
from tabulate import tabulate

In [99]:
results = [
    ['Logistic Regression','0.840'],
    ['Decision Tree', '0.774'],
    ['Random Forest', '0.844'],
    ['Gradient Boosting', '0.844'],
    ['SVM', 'error'],
    ['XGBoost', '0.841']
]

headers = ['Algorithm', 'accuracy_score']

table = tabulate(results, headers=headers, tablefmt='fancy_grid', floatfmt='.3f')

print(table)

╒═════════════════════╤══════════════════╕
│ Algorithm           │ accuracy_score   │
╞═════════════════════╪══════════════════╡
│ Logistic Regression │ 0.840            │
├─────────────────────┼──────────────────┤
│ Decision Tree       │ 0.774            │
├─────────────────────┼──────────────────┤
│ Random Forest       │ 0.844            │
├─────────────────────┼──────────────────┤
│ Gradient Boosting   │ 0.844            │
├─────────────────────┼──────────────────┤
│ SVM                 │ error            │
├─────────────────────┼──────────────────┤
│ XGBoost             │ 0.841            │
╘═════════════════════╧══════════════════╛


# modellarni saqlaymiz

In [100]:
from joblib import dump

In [ ]:
dump(grBoost, 'Gradient boosting.joblib')
dump(RF_Clas, 'Random_Forest.joblib')

['Gradient boosting.joblib']

In [105]:
dump(DT_Clas, 'Decision_Tree.joblib')

['Decision_Tree.joblib']

In [106]:
with open("jadval.csv", "w", encoding="utf-8") as f:
    f.write(table)